In [1]:
import os 
import numpy as np 
import pandas as pd 
import cv2 
import matplotlib.pyplot as plt 
import random

from sklearn.model_selection import train_test_split

from keras.preprocessing.image import ImageDataGenerator
from PIL import Image

from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [2]:
import tensorflow as tf 
from tensorflow import keras 
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,MaxPooling2D,Flatten, Dropout,BatchNormalization
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator

## Collecting data from different files and storing it in same array 

In [3]:
os.listdir(r'D:\anaconda3\My_Projects\my_work_proj\Face Recognition\Face_image_data')

['deril',
 'deril_augmented',
 'mahesh',
 'mahesh_augmented',
 'nikhil',
 'nikhil_augmented',
 'test']

In [4]:
file_names = ['deril','mahesh','nikhil']

In [5]:
images_path = r'D:\anaconda3\My_Projects\my_work_proj\Face Recognition\Face_image_data'
images = []
labels = []

label = 0

for i in os.listdir(images_path):
    
    if i in file_names:
        
        image_path = images_path + '\\' +i

        for j in os.listdir(image_path):

            path = image_path + '\\' + j 

            img = Image.open(path).resize((112,112))
            img = np.asarray(img)#/255

            images.append(img)
            labels.append(label)

        label += 1

    
images_array = np.array(images)
labels_array = np.array(labels)

In [10]:
# Code to save the images
# np.save('colored_images',images_array)
# np.save('colored_labels',labels_array)

In [7]:
np.unique(labels_array)

# 0 = DEril 
# 1- Mahesh 
# 2 - Nikhil

array([0, 1, 2])

In [8]:
## Checking if all the images are of same shape
count = 0 
for i in range(len(images_array)):
    if images_array[i].shape != (112,112,3):
        
        count +=1

print(count)

0


In [9]:
images_array.dtype

dtype('uint8')

In [10]:
np.unique(labels_array)

array([0, 1, 2])

## Splitting the data into training and testing data

In [11]:
x_train, x_val, y_train, y_val = train_test_split(images_array,labels_array,test_size=0.2,random_state=10)

In [12]:
np.unique(y_train)

array([0, 1, 2])

In [14]:
# doing onehot encoding of output 
y_train = keras.utils.to_categorical(y_train)
y_val = keras.utils.to_categorical(y_val)

x_train = x_train/255 
x_val = x_val/255

In [6]:
# Only done if we are working with gray images
# images_array_train = np.expand_dims(images_array_train, axis=3)
# images_array_test = np.expand_dims(images_array_test, axis=3)

In [16]:
np.unique(y_train)

array([0., 1.], dtype=float32)

In [17]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy

In [21]:
FR_Model_1 = Sequential([
    Conv2D(filters=128, kernel_size=[5,5], activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2)),
    BatchNormalization(),
    
    Conv2D(filters=326, kernel_size=[3,3], activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2)),
    BatchNormalization(),
    
    Conv2D(filters=500, kernel_size=[3,3], activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2)),
    BatchNormalization(),
    
    Flatten(),
    Dense(500, activation='relu'),
    Dropout(0.2),
    Dense(1000,activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
    
])

FR_Model_1.compile(optimizer=Adam(), loss=CategoricalCrossentropy(),metrics=Accuracy())

In [40]:
# def add_noise(img):
#     '''Add random noise to an image'''
#     VARIABILITY = 50
#     deviation = VARIABILITY*random.random()
#     noise = np.random.normal(0, deviation, img.shape)
#     img = img + noise
#     np.clip(img, 0., 255.)
#     return img

datagen = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True,
    rescale=1./255,
#     preprocessing_function=add_noise,
    fill_mode='nearest')

In [42]:
FR_Model_1.fit(datagen.flow(x_train,y_train,batch_size=32),epochs=10,validation_data = (x_val,y_val))

Epoch 1/10
8/8 [==============================] - 36s 4s/step - loss: 5.9064 - accuracy: 0.0899 - val_loss: 89.8085 - val_accuracy: 0.3060
Epoch 2/10
8/8 [==============================] - 36s 4s/step - loss: 1.8282 - accuracy: 0.2697 - val_loss: 145.4282 - val_accuracy: 0.5082
Epoch 3/10
8/8 [==============================] - 36s 4s/step - loss: 2.5707 - accuracy: 0.3306 - val_loss: 268.9976 - val_accuracy: 0.5956
Epoch 4/10
8/8 [==============================] - 36s 4s/step - loss: 1.4848 - accuracy: 0.4550 - val_loss: 354.1330 - val_accuracy: 0.5956
Epoch 5/10
5/8 [=================>............] - ETA: 11s - loss: 2.0500 - accuracy: 0.5011

KeyboardInterrupt: 

## Transfer Learing 

In [32]:
from keras.applications.vgg16 import VGG16
from keras.models import Model

In [43]:
vgg16 = VGG16(input_shape=(112,112,3), weights='imagenet', include_top=False)

for layer in vgg16.layers:
    layer.trainable = False

add_model = Sequential()
add_model.add(Flatten(input_shape=vgg16.output_shape[1:]))

add_model.add(Dense(1024, activation='relu'))
add_model.add(Dense(1024, activation='relu'))


add_model.add(Dense(512, activation='relu'))

add_model.add(Dense(y_train.shape[1], activation='softmax'))

model_vgg16 = Model(inputs=vgg16.input, outputs=add_model(vgg16.output))

learning_rate = 0.0001
# def results(model):
#     adam = Adam(lr=learning_rate)

    
model_vgg16.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

model_vgg16.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 112, 112, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 112, 112, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 112, 112, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 56, 56, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 56, 56, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 56, 56, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 28, 28, 128)       0   

In [44]:
history = model_vgg16.fit(datagen.flow(x_train, y_train, batch_size=128), epochs=15,validation_data=(x_val, y_val))

Epoch 1/15
2/2 [==============================] - 19s 10s/step - loss: 1.9841 - accuracy: 0.3568 - val_loss: 2.6913 - val_accuracy: 0.3443
Epoch 2/15
2/2 [==============================] - 17s 10s/step - loss: 1.9644 - accuracy: 0.3320 - val_loss: 1.2366 - val_accuracy: 0.3934
Epoch 3/15
2/2 [==============================] - 17s 9s/step - loss: 1.2748 - accuracy: 0.2656 - val_loss: 1.1281 - val_accuracy: 0.3934
Epoch 4/15
2/2 [==============================] - 17s 10s/step - loss: 1.1524 - accuracy: 0.3029 - val_loss: 1.1320 - val_accuracy: 0.3443
Epoch 5/15
2/2 [==============================] - 17s 9s/step - loss: 1.1419 - accuracy: 0.3195 - val_loss: 1.1432 - val_accuracy: 0.2623
Epoch 6/15
2/2 [==============================] - 17s 9s/step - loss: 1.1036 - accuracy: 0.3734 - val_loss: 1.0769 - val_accuracy: 0.3934
Epoch 7/15
2/2 [==============================] - 17s 10s/step - loss: 1.1306 - accuracy: 0.3195 - val_loss: 1.0899 - val_accuracy: 0.3443
Epoch 8/15
2/2 [==============

In [9]:
FR_Model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 112, 112, 64)      4864      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 56, 56, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 56, 56, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 56, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 28, 128)      0         
 2D)                                                             
                                                        

In [4]:
FR_Model_1 = keras.models.load_model('first_model_facial_recognition')
FR_Model_1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 112, 112, 64)      4864      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 batch_normalization_9 (Batc  (None, 56, 56, 64)       256       
 hNormalization)                                                 
                                                                 
 conv2d_10 (Conv2D)          (None, 56, 56, 128)       73856     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 28, 28, 128)      0         
 g2D)                                                            
                                                      

In [10]:
y_val_pred = FR_Model_1.predict(x_val)

130/130 [==============================] - 21s 159ms/step


In [11]:
np.argmax(y_val_pred,axis=1)

array([4, 0, 1, ..., 4, 2, 5], dtype=int64)

In [12]:
np.argmax(y_val,axis=1)

array([4, 0, 1, ..., 4, 2, 5], dtype=int64)

In [23]:
confusion_matrix(np.argmax(y_val,axis=1),np.argmax(y_val_pred,axis=1))

array([[508,   0,   0,   0,   0,   0,   0,   0],
       [  0, 546,   0,   0,   0,   0,   0,   0],
       [  0,   0, 514,   0,   0,   0,   0,   0],
       [  0,   0,   6, 520,   0,   0,   0,   2],
       [  0,   0,   4,   0, 490,   0,   0,   0],
       [  0,   0,   0,   0,   0, 517,   0,   0],
       [  0,   0,   0,   0,   0,   0, 538,   0],
       [  1,   0,   0,   0,   0,   0,   0, 513]], dtype=int64)

In [13]:
accuracy_score(np.argmax(y_val,axis=1),np.argmax(y_val_pred,axis=1))

0.9122385188747295

In [14]:
pd.DataFrame(classification_report(np.argmax(y_val,axis=1),np.argmax(y_val_pred,axis=1), output_dict = True)).transpose()

,precision,recall,f1-score,support
0,1.000000,0.881890,0.937238,508.000000
1,0.994536,1.000000,0.997260,546.000000
2,0.718487,0.998054,0.835505,514.000000
3,0.996226,1.000000,0.998110,528.000000
4,1.000000,0.635628,0.777228,494.000000
5,1.000000,0.789168,0.882162,517.000000
6,0.832817,1.000000,0.908784,538.000000
7,0.907273,0.970817,0.937970,514.000000
accuracy,0.912239,0.912239,0.912239,0.912239
macro avg,0.931167,0.909445,0.909282,4159.000000


## Testing on unseen data 

In [15]:
images_path = r'D:\anaconda3\My_Projects\my_work_proj\Face Recognition\Face_image_data\test'
images_test= []

    
for j in os.listdir(images_path):

    path = images_path + '\\' + j 

    img = plt.imread(path)
    img = Image.fromarray(img, 'RGB').resize((112,112))
    img = np.array(img)

    images_test.append(img)
    
    
images_test_array = np.array(images_test)

In [16]:
os.listdir(r'D:\anaconda3\My_Projects\my_work_proj\Face Recognition\Face_image_data\test')

['1.jpg', '2.jpg', '3.jpg']

In [17]:
images_test_array = images_test_array/225

In [18]:
testing = FR_Model_1.predict(images_test_array)

1/1 [==============================] - 0s 38ms/step


In [19]:
np.argmax(testing,axis=1)

array([5, 1, 5], dtype=int64)